In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import re

from tqdm.notebook import tqdm
tqdm.pandas()

from IPython.core.display import Markdown,display, HTML, Latex
import qgrid

#import seaborn as sns
import matplotlib.patheffects as pe
%matplotlib inline

import pickle

In [3]:
labels_multi_nli = {0: 'contradiction', 1 : 'neutral', 2 : 'entailment'}

## Load Paraphrased dataset to check opposition of semantics

In [4]:
cur_model_outcome_dir = '../../dfs_generated/paraphrased/paws/cumulative_tech_term_ner_ic/'
cur_models_all_files =  glob.glob(os.path.join(cur_model_outcome_dir, '*.pkl'))

In [5]:
list_cur_itr_model_dfs = []
for cur_model_cur_file_name in cur_models_all_files:
    tmp_df = pd.read_pickle(cur_model_cur_file_name)
    tmp_df['file_name'] = os.path.basename(cur_model_cur_file_name)
    list_cur_itr_model_dfs.append(tmp_df)

In [6]:
df_cur_itr_model_all_res = pd.concat(list_cur_itr_model_dfs, axis=0, ignore_index=True)

In [7]:
df_cur_itr_model_all_res['file_name'].unique()

array(['paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_11_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_1_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_10_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_9_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_refute_to_gen_support_0_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_4_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_refute_to_gen_support_1_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_3_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTargetDirection.org_support_to_gen_refute_7_FT_concat_prev.pkl',
       'paws_base_no_fine_tune_ParaphraseTar

In [8]:
df_cur_itr_model_all_res['paraphrase_model'] = df_cur_itr_model_all_res['file_name'].progress_apply(lambda x: pd.Series(re.split('[_ .]',x)[0]))
df_cur_itr_model_all_res['fine_tune_direction'] = df_cur_itr_model_all_res['file_name'].progress_apply(lambda x: pd.Series('_'.join(re.split('[_ .]',x)[6:11])))
df_cur_itr_model_all_res['no_of_df_used_for_fine_tuned'] = df_cur_itr_model_all_res['file_name'].progress_apply(lambda x: pd.Series(re.split('[_ .]',x)[11]))

  0%|          | 0/109090 [00:00<?, ?it/s]

  0%|          | 0/109090 [00:00<?, ?it/s]

  0%|          | 0/109090 [00:00<?, ?it/s]

In [9]:
# df_inc_dec = df_cur_itr_model_all_res[ (df_cur_itr_model_all_res['org_claim'].str.contains('increase'))
#                         & (df_cur_itr_model_all_res['gen_claim'].str.contains('increase'))][['org_claim', 'gen_claim']]
#df_inc_dec = df_cur_itr_model_all_res.copy()

In [10]:
#df_inc_dec.shape

In [11]:
#df_inc_dec = df_inc_dec.groupby(['org_claim', 'gen_claim']).size().reset_index(name='Freq')[['org_claim', 'gen_claim']]

## Filter for technocal terms

In [12]:
loc_df_scispacy_sentence_word_unq_ner_abr_filtered ='../../dfs_generated/linguistic/df_scispacy_sentence_word_unq_ner_abr_filtered.pkl'

In [13]:
df_scispacy_sentence_word_unq_ner_abr_filtered = pd.read_pickle(loc_df_scispacy_sentence_word_unq_ner_abr_filtered)

In [14]:
def filter_and_replace_tech_term_paraphrased_claim(claim_paraphrased, claim_original):
    #claim_para_trimmed = re.sub('[^a-z]+', ' ', claim_paraphrased.lower())
    df_cur_sentence_word_unq_ner_abr_filtered = df_scispacy_sentence_word_unq_ner_abr_filtered[
        df_scispacy_sentence_word_unq_ner_abr_filtered['claim'] == claim_original
    ]
    for cur_term_row in df_cur_sentence_word_unq_ner_abr_filtered.itertuples(index=False):
        cur_term_row_formatted = r'\b' + re.escape(cur_term_row.ner_text) + r'\b'
        if cur_term_row_formatted not in claim_paraphrased:
            return False        
#         if cur_term_row.ner_text.lower() not in claim_paraphrased.lower():
#             return False
    return True

## Roberta Negetion Checker

- https://stackoverflow.com/questions/69374258/sentence-similarity-models-not-capturing-opposite-sentences
- https://github.com/pytorch/fairseq/tree/main/examples/roberta

In [15]:
import torch
from fairseq.data.data_utils import collate_tokens

2022-01-10 02:07:19 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [16]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [17]:
model_neg_checker_roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')

model_neg_checker_roberta.to('cuda')
#model_neg_checker_roberta.cuda()

model_neg_checker_roberta.eval() 

Using cache found in /home/qudratealahyratu/.cache/torch/hub/pytorch_fairseq_main
2022-01-10 02:07:20 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.mnli.tar.gz from cache at /home/qudratealahyratu/.cache/torch/pytorch_fairseq/7685ba8546f9a5ce1a00c7a6d7d44f7e748d22681172f0f391c3d48f487c801c.74e37d47306b3cc51c5f8d335022a392c29f1906c8cd9e9cd3446d7422cf55d8
2022-01-10 02:07:24 | INFO | fairseq.tasks.masked_lm | dictionary: 50264 types
2022-01-10 02:07:34 | INFO | fairseq.models.roberta.model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 8, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance':

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dr

In [18]:
def get_mlnli_label(org_claim, gen_claim):    
    tokens_sentences_org_gen = model_neg_checker_roberta.encode(org_claim, gen_claim)
    logprobs_sentences_org_gen = model_neg_checker_roberta.predict('mnli', tokens_sentences_org_gen)      
    cal_val_mlnli_org_gen = logprobs_sentences_org_gen.argmax(dim=1).item()
    cal_label_mlnli_org_gen = labels_multi_nli[cal_val_mlnli_org_gen]
    
    tokens_sentences_gen_org = model_neg_checker_roberta.encode(gen_claim, org_claim)
    logprobs_sentences_gen_org = model_neg_checker_roberta.predict('mnli', tokens_sentences_gen_org)      
    cal_val_mlnli_gen_org = logprobs_sentences_gen_org.argmax(dim=1).item()
    cal_label_mlnli_gen_org = labels_multi_nli[cal_val_mlnli_gen_org]    
    
    return pd.Series([cal_val_mlnli_org_gen, cal_label_mlnli_org_gen, cal_val_mlnli_gen_org, cal_label_mlnli_gen_org])

In [19]:
df_tiny_test = pd.DataFrame(
[{'org_claim' : 'Antidepressants increase the severity of migraines.',
  'gen_claim' : 'Antidepressants decrease the severity of migraines'},
 {'org_claim' : 'Antidepressants increase the severity of migraines.',
  'gen_claim' : 'Antidepressants causes serious migraines'},
 {'org_claim' : 'Antidepressants increase the severity of migraines.',
  'gen_claim' : 'Antidepressants is not an OTC drug.'},
  {'org_claim' : 'Antimicrobial agents are less effective due to the pressure of antimicrobial usage.',
  'gen_claim' : 'The antimicrobial drugs are less effective.'},
])

In [20]:

df_tiny_test[['mlnli_val_org_gen', 'mlnli_label_org_gen', 'mlnli_val_gen_org', 'mlnli_label_gen_org']] = df_tiny_test.progress_apply(lambda cur_row : get_mlnli_label (cur_row['org_claim'], cur_row['gen_claim']), axis=1)

  0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
HTML(df_tiny_test.to_html())

,org_claim,gen_claim,mlnli_val_org_gen,mlnli_label_org_gen,mlnli_val_gen_org,mlnli_label_gen_org
0,Antidepressants increase the severity of migraines.,Antidepressants decrease the severity of migraines,0,contradiction,0,contradiction
1,Antidepressants increase the severity of migraines.,Antidepressants causes serious migraines,2,entailment,2,entailment
2,Antidepressants increase the severity of migraines.,Antidepressants is not an OTC drug.,1,neutral,1,neutral
3,Antimicrobial agents are less effective due to the pressure of antimicrobial usage.,The antimicrobial drugs are less effective.,2,entailment,1,neutral


In [22]:
df_cur_itr_model_all_res[['mlnli_val_org_gen', 'mlnli_label_org_gen', 'mlnli_val_gen_org', 'mlnli_label_gen_org']] = df_cur_itr_model_all_res.progress_apply(lambda cur_row : get_mlnli_label (cur_row['org_claim'], cur_row['gen_claim']), axis=1)

  0%|          | 0/109090 [00:00<?, ?it/s]

In [23]:
df_cur_itr_model_all_res['mlnli_label_org_gen'].value_counts()

entailment       85424
contradiction    17260
neutral           6406
Name: mlnli_label_org_gen, dtype: int64

In [24]:
len(df_cur_itr_model_all_res[(df_cur_itr_model_all_res['mlnli_label_org_gen'] == 'entailment') &
                        (df_cur_itr_model_all_res['mlnli_label_gen_org'] == 'entailment')])

65851

In [25]:
df_cur_itr_model_all_res['passed_ner_abr_filter_ic'] = df_cur_itr_model_all_res.progress_apply(lambda x: filter_and_replace_tech_term_paraphrased_claim(x['gen_claim'], x['org_claim']), axis=1)

  0%|          | 0/109090 [00:00<?, ?it/s]

In [26]:
df_cur_itr_model_all_res['passed_ner_abr_filter_ic'].value_counts()

False    103273
True       5817
Name: passed_ner_abr_filter_ic, dtype: int64

In [27]:
df_cur_itr_model_all_res.shape

(109090, 38)

## Attack calc

In [28]:
def get_dataframes_by_majority_org_claim(df_all_paraphrased_org_claim):
    df_all_paraphrased_org_success = df_all_paraphrased_org_claim[df_all_paraphrased_org_claim['org_comment'] == 'success']
    
    # Select claims with majority
    df_paraphrased_org_support_major = df_all_paraphrased_org_success[
        df_all_paraphrased_org_success['org_count_support'] > df_all_paraphrased_org_success['org_count_refute']
    ]

    
    df_paraphrased_org_refute_major = df_all_paraphrased_org_success[
        df_all_paraphrased_org_success['org_count_support'] < df_all_paraphrased_org_success['org_count_refute']
    ]
    
    return df_paraphrased_org_support_major, df_paraphrased_org_refute_major, df_all_paraphrased_org_success


In [29]:
def get_df_succesfully_attacked_claim(df_paraphrased_support_major, df_paraphrased_refute_major):
    df_org_refute_gen_support = df_paraphrased_refute_major[
    df_paraphrased_refute_major['gen_count_support'] > df_paraphrased_refute_major['gen_count_refute']
    ]

    df_org_support_gen_refute = df_paraphrased_support_major[
        df_paraphrased_support_major['gen_count_support'] < df_paraphrased_support_major['gen_count_refute']
    ]
    
    return df_org_support_gen_refute, df_org_refute_gen_support

In [30]:
df_paraphrased_selected_model_full = df_cur_itr_model_all_res.copy()

In [31]:
df_paraphrased_org_support_major, df_paraphrased_org_refute_major, df_all_paraphrased_cur_model_org_success = get_dataframes_by_majority_org_claim(df_paraphrased_selected_model_full)


# Get successfullt attacked claims after paraphrased
df_org_support_gen_refute, df_org_refute_gen_support = get_df_succesfully_attacked_claim(df_paraphrased_org_support_major, df_paraphrased_org_refute_major)


In [32]:
df_org_support_gen_refute['attack_type'] = 'org_support_gen_refute' 
df_org_refute_gen_support['attack_type'] = 'org_refute_gen_support'

/home/qudratealahyratu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/qudratealahyratu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
df_all_attack = pd.concat([df_org_support_gen_refute, df_org_refute_gen_support])

In [34]:
len(df_all_attack)

12685

In [35]:
 df_all_attack_filtered = df_all_attack[(df_all_attack['passed_ner_abr_filter_ic'] == True) &
          (df_all_attack['mlnli_label_org_gen'] == 'entailment') &
                                       (df_all_attack['mlnli_label_gen_org'] == 'entailment')]

In [36]:
len(df_all_attack_filtered)

327

In [37]:
len(df_all_attack_filtered['org_claim'].value_counts().keys())

15

In [38]:
 df_all_attack_filtered_clm = df_all_attack_filtered[['org_claim', 'gen_claim', 'attack_type', 'fine_tune_direction',
                                                       'model', 'no_of_df_used_for_fine_tuned']]

In [39]:
loc_path_file_name = '../../dfs_generated/filter_applied/paws_tech_term_ner_ic/cumulative_tech_term_ner_ic'

In [40]:
df_all_attack_filtered_clm.to_csv(loc_path_file_name+'_all_filtered_attack_clm.csv')
with open(loc_path_file_name+'_all_filtered_attack.pkl', 'wb') as fp:
    pickle.dump(df_all_attack_filtered_clm, fp)
    
df_all_attack.to_pickle(loc_path_file_name+'_all_attack_filter_label.pkl')

In [41]:
print('Total rephrase = {:10} || # of unique original claim covered = {:10}'.format(len(df_cur_itr_model_all_res), 
                                                                           len(df_cur_itr_model_all_res['org_claim'].value_counts().keys())))

print('Total attack = {:10} || # of unique original claim covered = {:10}'.format(len(df_all_attack), 
                                                                                  len(df_all_attack['org_claim'].value_counts().keys())))

print('Attack with org entailment = {:10} || # of unique original claim covered = {:10}'.format(len(df_all_attack[df_all_attack['mlnli_label_org_gen'] == 'entailment']), 
                                                                                  len(df_all_attack[df_all_attack['mlnli_label_org_gen'] == 'entailment']['org_claim'].value_counts().keys())))

print('Attack with both entailment = {:10} || # of unique original claim covered = {:10}'.format(len(df_all_attack[(df_all_attack['passed_ner_abr_filter_ic'] == True) &
          (df_all_attack['mlnli_label_org_gen'] == 'entailment')]), len(df_all_attack[(df_all_attack['passed_ner_abr_filter_ic'] == True) &
          (df_all_attack['mlnli_label_org_gen'] == 'entailment')]['org_claim'].value_counts().keys())))

print('Attack with both entailment valid ner = {:10} || # of unique original claim covered = {:10}'.format(len(df_all_attack_filtered), 
                                                                                  len(df_all_attack_filtered['org_claim'].value_counts().keys())))

Total rephrase =     109090 || # of unique original claim covered =        340
Total attack =      12685 || # of unique original claim covered =        310
Attack with org entailment =       6587 || # of unique original claim covered =        259
Attack with both entailment =        521 || # of unique original claim covered =         17
Attack with both entailment valid ner =        327 || # of unique original claim covered =         15


##### /paws/cumulative/
Total rephrase =      55201 || # of unique original claim covered =        340

Total attack =      11440 || # of unique original claim covered =        322

Attack with org entailment =       5118 || # of unique original claim covered =        262

Attack with both entailment =        342 || # of unique original claim covered =         14

Attack with both entailment valid ner =        123 || # of unique original claim covered =         12

##### /paws/cumulative_07_threshold/
Total rephrase =      51796 || # of unique original claim covered =        340
Total attack =       6072 || # of unique original claim covered =        277
Attack with org entailment =       4242 || # of unique original claim covered =        242
Attack with both entailment =        327 || # of unique original claim covered =         15
Attack with both entailment valid ner =        136 || # of unique original claim covered =         13


##### /paws/cumulative_tech_term_ner_ic/
Total rephrase =     109090 || # of unique original claim covered =        340
Total attack =      12685 || # of unique original claim covered =        310
Attack with org entailment =       6587 || # of unique original claim covered =        259
Attack with both entailment =        521 || # of unique original claim covered =         17
Attack with both entailment valid ner =        327 || # of unique original claim covered =         15

##### /paws/cumulative/
Total rephrase =      55201 || # of unique original claim covered =        340

Total attack =      11440 || # of unique original claim covered =        322

Attack with org entailment =       5118 || # of unique original claim covered =        262

Attack with both entailment =        342 || # of unique original claim covered =         14

Attack with both entailment valid ner =        123 || # of unique original claim covered =         12

##### /paws/cumulative_07_threshold/
Total rephrase =      51796 || # of unique original claim covered =        340
Total attack =       6072 || # of unique original claim covered =        277
Attack with org entailment =       4242 || # of unique original claim covered =        242
Attack with both entailment =        327 || # of unique original claim covered =         15
Attack with both entailment valid ner =        136 || # of unique original claim covered =         13


##### /paws/cumulative_tech_term_ner_ic/
Total rephrase =     109090 || # of unique original claim covered =        340
Total attack =      12685 || # of unique original claim covered =        310
Attack with org entailment =       6587 || # of unique original claim covered =        259
Attack with both entailment =        521 || # of unique original claim covered =         17
Attack with both entailment valid ner =        327 || # of unique original claim covered =         15

In [ ]:
import re
temp = 'This is correct'
src_txt = r'\b' + re.escape(temp) + r'\b'
if re.search(src_txt, 'This is correct and'):
    print('correct')